# Choropleth maps using global COVID 19 data and Lockdown analysis of Italy and India

### Please note due to the size of the maps, the output is supressed, you can re-run the code to generate all the outputs

# Import libraries

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
import folium
from folium.plugins import HeatMap

# Import the dataset

In [ ]:
dataset_url = 'https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv'
df = pd.read_csv(dataset_url)

In [ ]:
# Check the data
df.head()

# Global spread of covid with Choropleth map using Plotly

In [ ]:
fig = px.choropleth(df , locations = 'Country', locationmode = 'country names', color = 'Confirmed'
                    ,animation_frame = 'Date')
fig.update_layout(title_text = 'Global spread of Covid19')
fig.show()

### If you hover along the map, you will be able to view the dates, country names along with the confirmed cases

# Saving the Choropleth map in HTML file

In [ ]:
import plotly.offline as py

# html file
py.offline.plot(fig, filename='C:/plotlyplots/Global_covid.html')

### This file can be saved in a location in C drive and can be uploaded to github. The file I have created can be viewed in this location " https://mythili7.github.io/ "

# Global spread of covid with Choropleth map using Folium

## First generate the base map, we will use "Stamen Terrain" tiles

In [ ]:
map = folium.Map(tiles = 'Stamen Terrain',min_zoom = 1.5)
display(map);

# Obtain the geodata¶

### geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data
geo data file: world-countries.json

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

# Generate Choropleth Map Layer

In [ ]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=df,
    columns=['Country', 'Confirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name = 'Total Confirmed COVID cases',
).add_to(map)
map

# Generate markers to denote confirmed and recovered cases

### Get the latitude and longitides of all countries

In [ ]:
coordinates = pd.read_csv('world_coordinates.csv')
coordinates

# Merge the cordinates with the covid data

In [ ]:
covid_df = pd.merge(df, coordinates, on = 'Country')

In [ ]:
# Check the merged data

covid_df

# Define the function for the markers¶

### Here we will use circular markers, you can also use other custom markers

In [ ]:
def plotDot(point):
    folium.CircleMarker(location = (point.latitude, point.longitude),
                       radius = 5,
                       weight = 2,
                       popup = [point.Country, point.Confirmed, point.Recovered],
                       fill_color = '#000000').add_to(map)

# Add the markers to the earlier map

In [ ]:
covid_df.apply(plotDot, axis = 1)
map.fit_bounds(map.get_bounds())
map

In [ ]:
# The map can be saved in html file

map.save('C:/plotlyplots/Global_covid_Folium.html')

# Infection rate by country
# Let us visualize how severe the infection is with a specific country

In [ ]:
df_italy= covid_df[covid_df.Country == 'Italy']
df_italy.head()

## Select specific columns

In [ ]:
df_italy = df_italy[['Date' , 'Confirmed']]
df_italy.tail()

## To calculate infection rate, we will take the first derivative of the column 'Confirmed'

In [ ]:
df_italy['Infection rate'] = df_italy['Confirmed'].diff()
df_italy.tail()

# Plot the infection rate for Italy

In [ ]:
px.line(df_italy, x = 'Date',y = 'Infection rate')

In [ ]:
df_italy['Infection rate'].max()

# Calculate infection rate across all the countries

In [ ]:
countries = list(df['Country'].unique())
countries

In [ ]:
countries = list(df['Country'].unique())
max_infection_rates = []
for i in countries:
    MIR = df[df.Country == i].Confirmed.diff().max()
    max_infection_rates.append(MIR)
print(max_infection_rates)

## Create a new dataframe

In [ ]:
df_MIR = pd.DataFrame()
df_MIR['Country'] = countries
df_MIR['Max Infection rate'] = max_infection_rates
df_MIR.head()

# Plot the max infection rate for all the countries

In [ ]:
px.bar(df_MIR, x = 'Country', y = 'Max Infection rate', color = 'Country', title = 'MIR', log_y = True)

# Lockdown effect in Italy

### Lockdown was imposed on Italy on 9Mar2020. We will examine the effect post 1 month

In [ ]:
italy_lockdown_start_date = '2020-03-09'
italy_lockdown_a_month_later = '2020-04-09'

In [ ]:
df_italy2['Infection rate'] = df_italy2.Confirmed.diff()
df_italy2.head()

## Visualize before and after lockdown

In [ ]:
fig = px.line(df_italy, x = 'Date', y = 'Infection rate', title = 'Before and after lockdown')
fig.add_shape(
    dict(
    type = 'line',
    x0 = italy_lockdown_start_date,
    y0 = 0,
    x1 = italy_lockdown_start_date,
    y1 = df_italy['Infection rate'].max(),
    line = dict(color='red',width =2)
)

)

fig.add_shape(
    dict(
    type = 'line',
    x0 = italy_lockdown_a_month_later,
    y0 = 0,
    x1 = italy_lockdown_a_month_later,
    y1 = df_italy['Infection rate'].max(),
    line = dict(color='purple',width =2)
)

)

#### Lockdown had some effect in Italy, with the cases significantly decreasing during as well as after the lockdown.In recent times though, Sep 30 onwards, we again observe a surge in cases

# Impact on death rates

In [ ]:
df_italy2['Death rate'] = df_italy2.Deaths.diff()

In [ ]:
df_italy2.head()

In [ ]:
fig = px.line(df_italy2, x = 'Date', y = ['Infection rate', 'Death rate'])
fig.show()

#### Due to the scale, we are unable to observe the difference in the Infection rate and Death rate. Hence, we need to use data normalization

# Normalize the columns

In [ ]:
df_italy2['Infection rate']/df_italy2['Infection rate'].max()

In [ ]:
df_italy2['Infection rate'] = df_italy2['Infection rate']/df_italy2['Infection rate'].max()

In [ ]:
df_italy2['Death rate'] = df_italy2['Death rate']/df_italy2['Death rate'].max()

In [ ]:
fig = px.line(df_italy2, x = 'Date', y = ['Infection rate', 'Death rate'])

fig.show()

#### At the initial phases, the death rate was much higher than the infection rate. The death rate has stabilized around July'20, with a surge observed in the recent rise of COVID-19 cases. However, the death rate now is lower than the infection rates.

# Effect of lockdown in India

In [ ]:
india_lockdown_phase1 = '2020-03-25'
india_lockdown_phase2 = '2020-04-15'
india_lockdown_phase3 = '2020-05-04'
india_lockdown_phase4 = '2020-05-18'

In [ ]:
df_india = df[df.Country == 'India']
df_india

In [ ]:
df_india['Infection rate'] = df_india.Confirmed.diff()
df_india.head()

In [ ]:
fig = px.line(df_india, x = 'Date', y = 'Infection rate', title = 'Before and after lockdown')
fig.add_shape(
    dict(
    type = 'line',
    x0 = india_lockdown_phase1,
    y0 = 0,
    x1 = india_lockdown_phase1,
    y1 = df_india['Infection rate'].max(),
    line = dict(color='red',width =2)
)

)

fig.add_shape(
    dict(
    type = 'line',
    x0 = india_lockdown_phase2,
    y0 = 0,
    x1 = india_lockdown_phase2,
    y1 = df_india['Infection rate'].max(),
    line = dict(color='green',width =2)
)

)



fig.add_shape(
    dict(
    type = 'line',
    x0 = india_lockdown_phase3,
    y0 = 0,
    x1 = india_lockdown_phase3,
    y1 = df_india['Infection rate'].max(),
    line = dict(color='orange',width =2)
)

)



fig.add_shape(
    dict(
    type = 'line',
    x0 = india_lockdown_phase4,
    y0 = 0,
    x1 = india_lockdown_phase4,
    y1 = df_india['Infection rate'].max(),
    line = dict(color='purple',width =2)
)

)

#### For India, probably the lockdown came too soon and the there seems to be no effect felt of the lockdown. Post middle of Sep, the peak was reached and we can see a declining trend.